In [ ]:
!nvidia-smi

Sun Aug 27 09:30:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    42W / 300W |   7744MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip3 install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=cb7563f5e793154271a5e8406fc481b46354dbf112153a5ce3fa12460f

In [3]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from transformers import BlipProcessor, BlipForConditionalGeneration
from Data import CS10KDataset, MixDataset
import torch
import matplotlib.pyplot as plt
import re
from rouge_score import rouge_scorer
from tqdm import tqdm
import numpy as np
import pickle

In [4]:
#DATASET = "CS10K"
DATASET = "Mix"
DATA_DIR = 'drive/MyDrive/Multimodal-Image-Captioning/Data/'+DATASET+'/'
NUM_BATCHES = 400
RESIZE_SHAPE = None
VERBOSE = False
NUM_WORKERS = 10

if DATASET == "CS10K":
    dataset = CS10KDataset(DATA_DIR, NUM_BATCHES, RESIZE_SHAPE, VERBOSE, NUM_WORKERS)
else:
    dataset = MixDataset(DATA_DIR, NUM_BATCHES, RESIZE_SHAPE, VERBOSE, NUM_WORKERS)
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

In [ ]:
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16).cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large", truncation_side="left")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").cuda()

In [6]:
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained('drive/MyDrive/Multimodal-Image-Captioning/model_qlora_mix_2', torch_dtype=torch.float16).cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model

In [7]:
def show_img(img):
    plt.imshow(img)
    plt.show()

def get_prefix(data_point,typ):
    if typ == 0:
        prefix = f'Question: What is the caption for this Figure?\nAnswer: '
    elif typ == 1:
        prefix = f'Question: What is the caption for this Figure with Title: "{data_point["title"]}"?\nAnswer: '
    else:
        prefix = f'Question: What is the caption fro this Figure with Title: "{data_point["title"]}" and Abstract: "{data_point["abstract"]}"?\nAnswer: '
    return prefix

def avg(l):
    return sum(l) / len(l)


In [8]:
rouge1 = []
rouge2 = []
rougeL = []
f1score = 0
outputs = []
titles = []
abstracts = []

import random
random.seed(42)

model.eval()
for batch in range(60):
    print(f'Batch {batch}')
    batch = dataset[len(dataset) - batch - 1]
    loop = tqdm(batch)
    for idx, item in enumerate(loop):
        try:
            TYPE = random.choice([0,1,2])
            loop.set_description(f'Preparing Inputs\tRouge Score: {f1score}')
            image = item['figure']
            if len(image.shape) ==2:
                image = np.dstack((image, image, image))
            label = item['label']
            text = get_prefix(item,TYPE)
            inputs = processor(image, text, return_tensors="pt").to("cuda",torch.float16)
            loop.set_description(f'Running Model\tRouge Score: {f1score}')
            output = processor.decode(model.generate(**inputs, temperature=0.95, max_new_tokens=len(label), do_sample=True, top_k=50)[0], skip_special_tokens=True)
            try:
                index = output.index('Question:')
                output = output[:index]
            except:
                pass
            loop.set_description(f'Calculating Score\tRouge Score: {f1score}')
            score = scorer.score(label.lower(), output.lower())
            outputs.append((label,output))
            titles.append(item['title'])
            abstracts.append(item['abstract'])
            rouge1.append(score['rouge1'].fmeasure)
            rouge2.append(score['rouge2'].fmeasure)
            rougeL.append(score['rougeL'].fmeasure)
            f1score = rouge1[-1]
        except:
            print(f'Error')

f1_scores = {'rouge1': rouge1, 'rouge2': rouge2, 'rougeL': rougeL, 'outputs':outputs,'titles':titles,'abstracts':abstracts}
with open('drive/MyDrive/Multimodal-Image-Captioning/model_mix.pkl','wb') as f:
    pickle.dump(f1_scores,f)

Batch 0


Calculating Score	Rouge Score: 0.07194244604316546: 100%|██████████| 38/38 [03:02<00:00,  4.82s/it]


Batch 1


Calculating Score	Rouge Score: 0.036036036036036036: 100%|██████████| 36/36 [02:45<00:00,  4.61s/it]


Batch 2


...

Running Model	Rouge Score: 0.048780487804878044:  71%|███████   | 49/69 [04:13<02:22,  7.13s/it]   

Error


Running Model	Rouge Score: 0.3010752688172043:  74%|███████▍  | 51/69 [04:20<01:37,  5.40s/it]   

Error
Error
Error
Error
Error


Calculating Score	Rouge Score: 0.1348314606741573: 100%|██████████| 69/69 [05:42<00:00,  4.96s/it]
